In [5]:
%pwd

'c:\\Users\\Jaisal\\OneDrive\\Desktop\\New_me\\Langchain_Projects\\Medical-Chatbot\\research'

In [6]:
import os
os.chdir('../')

In [7]:
%pwd

'c:\\Users\\Jaisal\\OneDrive\\Desktop\\New_me\\Langchain_Projects\\Medical-Chatbot'

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Extract the data from the pdf file

In [9]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                              glob="*.pdf",
                              loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [10]:
extracted_data = load_pdf_file(data = 'data/')

In [11]:

# Split the documents into smaller chunks

def text_splitter(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [12]:
text_chunks = text_splitter(extracted_data)
print(f"Total number of chunks: {len(text_chunks)}")

Total number of chunks: 5699


In [13]:
# Download the embeddings from Hugging Face
from langchain.embeddings import HuggingFaceEmbeddings


In [14]:

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

In [15]:

embeddings = download_hugging_face_embeddings()

C:\Users\Jaisal\AppData\Local\Temp\ipykernel_21604\3649822238.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [20]:
query = embeddings.embed_query("What is the capital of France?")
print(f"length of the query: {len(query)}")

length of the query: 384


In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [ ]:

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-db"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [23]:
index_name = "medical-db"
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [ ]:
'''from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)
'''
# Already loaded the data into Pinecone, uncomment the above code to load the data into Pinecone if you are running this notebook for the first time

In [25]:
# Loading the existing index

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [27]:
retrieved_docs = retriever.invoke("What is types of diabeties?")

## Checking whether our query can be answered by retrieving



In [28]:
retrieved_docs

[Document(id='5688ae4a-8f0b-4116-bfd5-ca59fba4f1ea', metadata={'author': 'Clifford', 'creationdate': '2004-12-28T15:38:25-05:00', 'creator': 'PyPDF', 'enhanced': 'By PDF Enhancer 2.5/Win', 'moddate': '2005-05-04T13:53:15-06:00', 'page': 47.0, 'page_label': '48', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Med_book.pdf', 'spdf': '1096', 'total_pages': 599.0}, page_content='stop responding to the insulin that is produced, so that\nglucose in the blood cannot be absorbed into the cells of\nthe body . Symptoms include frequent urination, tired-\nness, excessive thirst, and hunger.\nDescription\nDiabetes mellitus is a chronic disease that causes se-\nrious health complications including renal (kidney) fail-\nure, heart disease , stroke, and blindness. Approximate-\nly 14 million Americans (about 5% of the population)\nhave diabetes. Unfortunately , as many as one-half of'),
 Document(id='19d3abbd-0251-400e-b340-60b7b242e82d', metadata={'author': 'Clifford', 'creationdate': '200

The answer is retrieved, but we need to make it more readable form for the user

## Initializing the LLM

In [ ]:
import os
HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


### Flan T5 Base as LLM

Since, I am running this code in my low end laptop with only CPU, I preferred a smaller model for the LLM task. You can freely replace this with larger models or even the premium ones like OpenAI GPTs.

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline

model_id = "google/flan-t5-base"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create Hugging Face pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    device_map="auto"  # Uses GPU if available, else CPU
)

# Wrap the pipeline with LangChain
llm = HuggingFacePipeline(pipeline=pipe)


c:\Users\Jaisal\anaconda3\envs\medbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jaisal\anaconda3\envs\medbot\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jaisal\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate d

In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a helpful assistant that provides information about diabetes. "
    "You will be provided with some context and you need to answer the question based on the context.Use three sentences maximum and keep the answer short. "
    "If the context does not contain the answer, say 'I don't know'. "
    "If the context is too long, summarize it in three sentences maximum. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [30]:
combine_docs_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [40]:
response = rag_chain.invoke({"input": "What is diabetes?"})
print(response["answer"])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


A group of metabolic disorders in which the body produces insufficient insulin or is resistant to the insulin it does produce, causing glucose levels to rise in the blood.
